In [15]:
import sys
sys.path.append("..")
import random
from sklearn.model_selection import train_test_split
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation

from movingpose.estimator import neighbors
from movingpose.estimator import classifiers

from movingpose.preprocessing import moving_pose
from movingpose.preprocessing import kinect_skeleton_data

## Pickle multiview action data

In [16]:
kinect_skeleton_data.pickle_dir("../pickle/multiview.p", "../ext/dataset/multiview_action/")

Pickling skeleton data...
Saving data...
Saved data to file: ../pickle/multiview.p
Program took 15.321957111358643 to run!


## Load pickled multiview action data

In [17]:
raw_data_dict = kinect_skeleton_data.load_pickle("../pickle/multiview.p")

## Format multiview action data

In [18]:
X, labels = moving_pose.format_skeleton_data_dict(raw_data_dict)

## Visualize formatted skeleton data

In [19]:
# initialize joint colors of skeleton
colors = ['blue' for i in range(20)]
colors[0] = 'red'  # hip is red
colors[7] = 'green'  # hands and feet are green
colors[11] = 'green'
colors[15] = 'green'
colors[19] = 'green'
colors[3] = 'black' # head is black

action_data = X[random.randrange(0, len(X))]

print(action_data)

# get array of all frames without joint number or frame number
all_frames = [np.array(i)[:,2:] for i in action_data]
max_frame = len(all_frames)

def update_graph(frame_data):
    xs, ys, zs = frame_data[:,0], frame_data[:,1], frame_data[:,2]
    graph._offsets3d = (xs, ys, zs)
#     p1 = [xs[2], ys[2], zs[2]]
#     p2 = [xs[4], ys[4], zs[4]]
#     temp = plt.plot(p1, p2)
    return graph

fig = plt.figure()
#fig.set_size_inches(9.5, 9.5)
ax = fig.add_subplot(111, projection='3d')
ax.view_init(-75,  90)
points = all_frames[0]
graph = ax.scatter(points[:,0], points[:,1], points[:,2], color=colors)

#xs, ys, zs = all_frames[0][:,0], all_frames[0][:,1], all_frames[0][:,2] ; print(xs)

ani = matplotlib.animation.FuncAnimation(fig, update_graph, all_frames, interval=90)


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2], [-0.008803109089408998, 0.029343696964696655, 0.06357801009017598, -0.0009603229766018025, -5.340076666425497e-05, -0.00011570166110588809, 6.025012380813863e-05, -0.0011722625199827794, 0.0005362889908971935, 2], [-0.02414837945734874, 0.32294987000461006, 0.181225082911047, -0.000960322976601799, -5.340076666421334e-05, -0.00011570166110588809, 0.002161837390107134, -0.0012921672383480187, 0.001098870655770967, 2], [-0.022129838463555217, 0.524803969383962, 0.21352173881174333, 4.813475428851535e-05, -0.00021595396795381916, 0.0008683730895908959, 0.0011549312181973817, -0.000974458139002321, -0.0008704254276241108, 2], [-0.17070188680285003, 0.20277599398129903, 0.12260367997284644, -0.0005189057231401728, -0.0004896892027405619, -0.00032852528846021023, 0.0019350025566748918, -0.0006799212178476011, 0.0004219344183002405, 2], [-0.2626213239168073, 0.002224494823574036, 0.030684242858889516, 0.00317383469133381, -0.00096616697795295

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## Create train/test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, random_state=42)

## Train Action Classifier with normalized training data

In [ ]:
best_score = sys.maxsize
best_action_classifier = None
for n_neighbors in range(5, 15, 5):
    for n_training_neighbors in range(10, 40, 10):
        for alpha in np.arange(0.2, 0.8, 0.1):
            for beta in np.arange(0.1, 0.5, 0.1):
                for kappa in range(3, 10, 1):
                    nearest_descriptor_estimator = neighbors.NearestPoses(
                        n_neighbors=n_neighbors,
                        n_training_neighbors=n_training_neighbors,
                        alpha=alpha,
                        beta=beta,
                        kappa=kappa
                    )
                    for theta in np.arange(0.2, 0.8, 0.05):
                        for n in range(5, 15, 1):
                            action_classifier = classifiers.ActionClassifier(
                                nearest_descriptor_estimator=nearest_descriptor_estimator,
                                theta=theta,
                                n=n
                            )
                            action_classifier.fit(X_train, y_train)
                            y_pred = action_classifier.predict(X_test)

                            diff = 0
                            for pred, actual in zip(y_pred, y_test):
                                diff += abs(pred - actual)

                            if diff < best_score:
                                best_score = diff
                                best_action_classifier = action_classifier

## Print best action classifier and its score

In [ ]:
print(f"The best action classifier was {best_action_classifier}")
print("\n--------------\n")
print(f"Its score was: {best_score}")

## Save the best action classifier

In [ ]:
best_action_classifier.save_pickle("../pickle/best_action_classifier.p")

## Load best action classifier

In [ ]:
action_classifier = classifiers.load_pickle("../pickle/best_action_classifier.p")

## Predict live data

In [ ]:
#TODO add live predictions when mehmet publishes the UI code

# DAB

In [ ]:
print("dab")
